In [1]:
#pip install KaggleDatasetAdapter

In [2]:
import kagglehub
from kagglehub import KaggleDatasetAdapter

# Set the path to the file you'd like to load (update this if needed)
file_path = "WA_Fn-UseC_-Telco-Customer-Churn.csv"

# Load the latest version of the dataset as a Pandas DataFrame
df = kagglehub.load_dataset(KaggleDatasetAdapter.PANDAS, "blastchar/telco-customer-churn", file_path)
df

C:\Users\learning\AppData\Local\Temp\ipykernel_7944\1075883708.py:8: DeprecationWarning: load_dataset is deprecated and will be removed in a future version.
  df = kagglehub.load_dataset(KaggleDatasetAdapter.PANDAS, "blastchar/telco-customer-churn", file_path)


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,6840-RESVB,Male,0,Yes,Yes,24,Yes,Yes,DSL,Yes,...,Yes,Yes,Yes,Yes,One year,Yes,Mailed check,84.80,1990.5,No
7039,2234-XADUH,Female,0,Yes,Yes,72,Yes,Yes,Fiber optic,No,...,Yes,No,Yes,Yes,One year,Yes,Credit card (automatic),103.20,7362.9,No
7040,4801-JZAZL,Female,0,Yes,Yes,11,No,No phone service,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.60,346.45,No
7041,8361-LTMKD,Male,1,Yes,No,4,Yes,Yes,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Mailed check,74.40,306.6,Yes


In [3]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

In [4]:
print(df.columns)

Index(['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents',
       'tenure', 'PhoneService', 'MultipleLines', 'InternetService',
       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
       'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
       'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn'],
      dtype='object')


In [5]:
def assign_churn_risk(row):
    score = 0

    # Low tenure = higher risk
    if row['tenure'] < 6:
        score += 2
    elif row['tenure'] < 12:
        score += 1

    # High monthly charges = higher risk
    if row['MonthlyCharges'] > 80:
        score += 2
    elif row['MonthlyCharges'] > 60:
        score += 1

    # Month-to-month contracts are riskier
    if row['Contract'] == 'Month-to-month':
        score += 2

    # Paperless billing often correlates with higher churn
    if row['PaperlessBilling'] == 'Yes':
        score += 1

    return score


In [6]:
df['churn_risk_score'] = df.apply(assign_churn_risk, axis=1)

In [7]:
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

In [8]:
df.fillna({'TotalCharges': df['TotalCharges'].median()}, inplace=True)

In [9]:
# List of desired columns to encode
to_encode = ['gender', 'InternetService', 'Contract', 'PaymentMethod']

# Filter only those that exist in the DataFrame
existing_cols = [col for col in to_encode if col in df.columns]

# Apply get_dummies only to existing columns
df = pd.get_dummies(df, columns=existing_cols, drop_first=True)
df.head()

,customerID,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,OnlineSecurity,OnlineBackup,DeviceProtection,...,Churn,churn_risk_score,gender_Male,InternetService_Fiber optic,InternetService_No,Contract_One year,Contract_Two year,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
0,7590-VHVEG,0,Yes,No,1,No,No phone service,No,Yes,No,...,No,5,False,False,False,False,False,False,True,False
1,5575-GNVDE,0,No,No,34,Yes,No,Yes,No,Yes,...,No,0,True,False,False,True,False,False,False,True
2,3668-QPYBK,0,No,No,2,Yes,No,Yes,Yes,No,...,Yes,5,True,False,False,False,False,False,False,True
3,7795-CFOCW,0,No,No,45,No,No phone service,Yes,No,Yes,...,No,0,True,False,False,True,False,False,False,False
4,9237-HQITU,0,No,No,2,Yes,No,No,No,No,...,Yes,6,False,True,False,False,False,False,True,False


In [10]:
df.to_csv("cleaned_data.csv", index=False)

In [11]:
pip install snowflake-connector-python pandas

   ---------------------------------------- 0.0/947.3 kB ? eta -:--:--
   --------------------------------- ------ 786.4/947.3 kB 4.2 MB/s eta 0:00:01
   ---------------------------------------- 947.3/947.3 kB 4.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/13.5 MB ? eta -:--:--
   --- ------------------------------------ 1.0/13.5 MB 5.6 MB/s eta 0:00:03
   ---- ----------------------------------- 1.6/13.5 MB 4.9 MB/s eta 0:00:03
   ------ --------------------------------- 2.1/13.5 MB 3.2 MB/s eta 0:00:04
   ------ --------------------------------- 2.4/13.5 MB 2.9 MB/s eta 0:00:04
   -------- ------------------------------- 2.9/13.5 MB 2.5 MB/s eta 0:00:05
   -------- ------------------------------- 2.9/13.5 MB 2.5 MB/s eta 0:00:05
   --------- ------------------------------ 3.1/13.5 MB 2.3 MB/s eta 0:00:05
   ---------- ----------------------------- 3.7/13.5 MB 2.1 MB/s eta 0:00:05
   ------------ --------------------------- 4.2/13.5 MB 2.1 MB/s eta 0:00:05
   ---

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.12.3 requires botocore<1.34.70,>=1.34.41, but you have botocore 1.38.10 which is incompatible.


In [ ]:
import pandas as pd
import snowflake.connector
import os
from dotenv import load_dotenv

# Data load
df = pd.read_csv("F:\\CDE\\Projects\\Telecom ETL\\cleaned_data.csv")

load_dotenv()

# Access Snowflake credentials from environment
conn = snowflake.connector.connect(
    user=os.getenv("SNOWFLAKE_USER"),
    password=os.getenv("SNOWFLAKE_PASSWORD"),
    account=os.getenv("SNOWFLAKE_ACCOUNT"),
    warehouse=os.getenv("SNOWFLAKE_WAREHOUSE"),
    database=os.getenv("SNOWFLAKE_DATABASE"),
    schema=os.getenv("SNOWFLAKE_SCHEMA"),
    role=os.getenv("SNOWFLAKE_ROLE")
)

cursor = conn.cursor()

# Ensure schema is set
cursor.execute("USE DATABASE telecom_db")
cursor.execute("USE SCHEMA telecom_db.churn_schema")

# Create table
columns = ", ".join([f'"{col}" STRING' for col in df.columns])
create_stmt = f'CREATE OR REPLACE TABLE telecom_db.churn_schema.churn_data ({columns})'
print("Create Statement:", create_stmt)
cursor.execute(create_stmt)

# Verify table creation
try:
    cursor.execute("SELECT * FROM telecom_db.churn_schema.churn_data")
    print("Table created successfully!")
except snowflake.connector.errors.ProgrammingError as e:
    print(f"Error while verifying table: {e}")

# Manually upload file to stage and copy into table
try:
    # Put file into Snowflake stage (using single quotes with escaped path)
    cursor.execute("PUT 'file:///F:/CDE/Projects/Telecom ETL/cleaned_data.csv' @%churn_data")
    print("File uploaded to stage successfully!")

    # Copy data into table
    cursor.execute("COPY INTO telecom_db.churn_schema.churn_data FROM @%churn_data FILE_FORMAT = (TYPE = CSV SKIP_HEADER = 1)")
    print("✅ Data inserted successfully!")
except snowflake.connector.errors.ProgrammingError as e:
    print(f"Error while inserting data: {e}")

cursor.close()
conn.close()

Create Statement: CREATE OR REPLACE TABLE telecom_db.churn_schema.churn_data ("customerID" STRING, "SeniorCitizen" STRING, "Partner" STRING, "Dependents" STRING, "tenure" STRING, "PhoneService" STRING, "MultipleLines" STRING, "OnlineSecurity" STRING, "OnlineBackup" STRING, "DeviceProtection" STRING, "TechSupport" STRING, "StreamingTV" STRING, "StreamingMovies" STRING, "PaperlessBilling" STRING, "MonthlyCharges" STRING, "TotalCharges" STRING, "Churn" STRING, "churn_risk_score" STRING, "gender_Male" STRING, "InternetService_Fiber optic" STRING, "InternetService_No" STRING, "Contract_One year" STRING, "Contract_Two year" STRING, "PaymentMethod_Credit card (automatic)" STRING, "PaymentMethod_Electronic check" STRING, "PaymentMethod_Mailed check" STRING)
Table created successfully!
File uploaded to stage successfully!
✅ Data inserted successfully!
